<a href="https://colab.research.google.com/github/wanutchapornmun/229352-StatisticalLearning/blob/main/Lab02_Data_Preprocessing_660510561.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [ ]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"], axis=1) # ลบ (drop) column (axis=1) ออก
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [ ]:
print("--- Missing Values Count ---")
print(df.isnull().sum()) # ตรวจสอบจำนวนของ Missing Values ว่ามีจำนวนเท่าไหร่ ในแต่ละ column โดยใช้การ sum เข้ามาช่วย

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [ ]:
print("--- Unique Values for Categorical Columns ---") # คำสั่งที่เอาไว้ดูค่าที่เป็นไปได้ของแต่ละตัวแปร
for col in df.select_dtypes(include='object').columns: # จะเลือกทุกๆ col ที่ชนิดข้อมูลเป็น obj
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts (บอกค่าที่หายไป (NAN) ด้วย )

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

ขั้นตอนก่อนที่จะนำไปสร้าง model

In [ ]:
# Map target variable 'y' to 0 (no) and 1 (yes) ; model ส่วนใหญ่ไม่ค่อยรองรับข้อมูลที่เป็น text ดังนั้น ต้องแปลงให้เป็น ตัวเลข (แปลง y ให้เป็น ตัวเลข)
df['y_new'] = df['y'].map({'yes':1, 'no':0}) # ถ้าตอบ yes แปลงเป็น 1 และถ้าตอบ no --> 0


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
X = df.drop(['y', 'y_new'], axis=1) # ใช้คำสั่งนี้ดูที่แกนตั้ง (axis)
# Split the data BEFORE any transformations
#X_train, X_test, y_test =train_test_split(X, y, test_size=0.3)

# Print data shape
print(y)
print(X)




0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      n

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train, X_test, y_train, y_test # เป็นการแบ่งข้อมูลเป็นชุดฝึก (Training) และชุดทดสอบ (Test) ก่อนนำไปสร้างโมเดล

(       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
 34695   26   technician   single  professional.course      no     yes   no   cellular   may         thu       228         1    999         0  nonexistent
 7667    47       admin.  married          high.school      no     yes   no  telephone   may         fri       352         3    999         0  nonexistent
 38441   30       admin.   single    university.degree      no      no  yes   cellular   oct         thu       102         1    999         1      failure
 2368    32          NaN  married             basic.6y     NaN      no   no  telephone   may         tue        39         2    999         0  nonexistent
 6672    31  blue-collar  married             basic.9y      no      no   no  telephone   may         wed        81         4    999         0  nonexistent
 ...    ...          ...      ...                  ...     ...     ...

We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [ ]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data
X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
28896,-0.571825,technician,married,professional.course,no,no,no,cellular,apr,fri,141,-0.203395,999,0,nonexistent
9487,1.343692,blue-collar,NaN,basic.6y,NaN,no,no,telephone,jun,mon,832,5.567270,999,0,nonexistent
34559,-1.146480,admin.,married,high.school,no,NaN,NaN,cellular,may,thu,430,0.517938,999,0,nonexistent
11800,-0.954928,blue-collar,married,basic.4y,NaN,yes,no,telephone,jun,fri,83,0.878605,999,0,nonexistent
5235,-0.380273,blue-collar,married,basic.6y,NaN,no,no,telephone,may,fri,378,-0.203395,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3380,0.960589,services,single,high.school,NaN,yes,no,telephone,may,thu,468,-0.203395,999,0,nonexistent
19448,-0.284497,technician,married,high.school,NaN,no,no,cellular,aug,thu,91,-0.564061,999,0,nonexistent
34570,-0.284497,services,divorced,high.school,no,no,no,cellular,may,thu,36,-0.564061,999,0,nonexistent
27269,1.439468,retired,married,high.school,no,no,no,cellular,nov,fri,532,-0.564061,999,0,nonexistent


In [ ]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-2.750393e-16,257.926746,-4.460762e-17,961.906073,0.171101
std,1.000017e+00,258.560909,1.000017e+00,188.307931,0.491374
min,-2.200015e+00,0.000000,-5.640615e-01,0.000000,0.000000
25%,-7.633766e-01,102.000000,-5.640615e-01,999.000000,0.000000
50%,-1.887214e-01,180.000000,-2.033949e-01,999.000000,0.000000
75%,6.732614e-01,320.000000,1.572717e-01,999.000000,0.000000
max,5.557831e+00,4199.000000,1.927260e+01,999.000000,7.000000


Let's take a look at the transformed `age` and `campaign` features and their statistics.

**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation** # การเติมค่าที่หายไป (missing)

---



In [ ]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

# แปลง training set และ test set เพื่อเติมค่าที่หายไป
X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
34695,professional.course
7667,high.school
38441,university.degree
2368,basic.6y
6672,basic.9y
...,...
19286,university.degree
1004,university.degree
39040,university.degree
14125,high.school


- **Ordinal Encoding**

In [ ]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [ ]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories]) # ordinal ของ sci-kit learn

X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])

Let's take a look at the imputed and ordinal-encoded `education`.

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [ ]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
#imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo] = imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [ ]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot = onehot_encoder.transform(X_test[['job']])

X_train_onehot.shape

(28831, 11)

In [ ]:
X_train_onehot

array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [ ]:
X_train = pd.concat([X_train.reset_index(drop=True),
                     pd.DataFrame(X_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
X_train = X_train.drop(['job'], axis=1) # ข้อมูลสุดท้ายที่ได้จะต้องเป็นตัวเลขเท่านั้น

X_train

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-1.338032,single,5.0,no,yes,no,cellular,may,thu,228,-0.564061,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.673261,married,4.0,no,yes,no,telephone,may,fri,352,0.157272,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.954928,single,6.0,no,no,yes,cellular,oct,thu,102,-0.564061,999,1,failure,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.763377,married,2.0,NaN,no,no,telephone,may,tue,39,-0.203395,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.859152,married,3.0,no,no,no,telephone,may,wed,81,0.517938,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.859152,single,6.0,no,yes,no,cellular,aug,wed,166,-0.564061,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,0.673261,married,6.0,no,NaN,NaN,telephone,may,wed,106,-0.564061,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,2.205675,married,6.0,no,yes,no,cellular,dec,mon,510,-0.203395,999,1,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28829,0.002830,single,4.0,no,yes,no,cellular,jul,mon,172,0.157272,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_test= pd.concat([X_test.reset_index(drop=True),
                     pd.DataFrame(X_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
X_test = X_test.drop(['job'], axis=1)

X_test

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.571825,married,5.0,no,no,no,cellular,apr,fri,141,-0.203395,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.343692,NaN,2.0,NaN,no,no,telephone,jun,mon,832,5.567270,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.146480,married,4.0,no,NaN,NaN,cellular,may,thu,430,0.517938,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.954928,married,1.0,NaN,yes,no,telephone,jun,fri,83,0.878605,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.380273,married,2.0,NaN,no,no,telephone,may,fri,378,-0.203395,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,0.960589,single,4.0,NaN,yes,no,telephone,may,thu,468,-0.203395,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12353,-0.284497,married,4.0,NaN,no,no,cellular,aug,thu,91,-0.564061,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12354,-0.284497,divorced,4.0,no,no,no,cellular,may,thu,36,-0.564061,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12355,1.439468,married,4.0,no,no,no,cellular,nov,fri,532,-0.564061,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


nominal_features --> แปลงโดยใช้ onehotencoder

In [ ]:
# ส่งงานแค่ exercise 1
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---

# 1. Numerical Feature: age, campaign, pdays, previous
num_feature = ['age', 'campaign', 'pdays', 'previous']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()
X_train_copy[num_feature] = scaler.fit_transform(X_train_copy[num_feature])
X_test_copy[num_feature] = scaler.transform(X_test_copy[num_feature])

X_train_copy

#X_test_copy

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-1.338032,single,5.0,no,yes,no,cellular,may,thu,228,-0.564061,0.196989,-0.348214,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.673261,married,4.0,no,yes,no,telephone,may,fri,352,0.157272,0.196989,-0.348214,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.954928,single,6.0,no,no,yes,cellular,oct,thu,102,-0.564061,0.196989,1.686930,failure,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.763377,married,2.0,NaN,no,no,telephone,may,tue,39,-0.203395,0.196989,-0.348214,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.859152,married,3.0,no,no,no,telephone,may,wed,81,0.517938,0.196989,-0.348214,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.859152,single,6.0,no,yes,no,cellular,aug,wed,166,-0.564061,0.196989,-0.348214,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,0.673261,married,6.0,no,NaN,NaN,telephone,may,wed,106,-0.564061,0.196989,-0.348214,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,2.205675,married,6.0,no,yes,no,cellular,dec,mon,510,-0.203395,0.196989,1.686930,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28829,0.002830,single,4.0,no,yes,no,cellular,jul,mon,172,0.157272,0.196989,-0.348214,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 2. Ordinal Feature: education
ord_feature = ['education']

X_train_copy[ord_feature] = df.loc[X_train_copy.index, ord_feature]
X_test_copy[ord_feature] = df.loc[X_test_copy.index, ord_feature]

imputer_ord = SimpleImputer(strategy='most_frequent')

X_train_copy[ord_feature] = imputer_ord.fit_transform(X_train_copy[ord_feature])
X_test_copy[ord_feature] = imputer_ord.transform(X_test_copy[ord_feature])

education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

ordinal_encoder = OrdinalEncoder(categories=[education_categories]) # ordinal ของ sci-kit learn

X_train_copy[ord_feature] = ordinal_encoder.fit_transform(X_train_copy[ord_feature])
X_test_copy[ord_feature] = ordinal_encoder.transform(X_test_copy[ord_feature])

X_train_copy[ord_feature]

,education
0,1.0
1,4.0
2,4.0
3,2.0
4,4.0
...,...
28826,3.0
28827,4.0
28828,1.0
28829,4.0


In [ ]:
# 3. Nominal Feature: job, marital, contact, month, day_of_week, poutcome, default, housing, loan
nominal_feature = ['marital','contact','month','day_of_week','poutcome','default','housing','loan']

imputer_nom = SimpleImputer(strategy='most_frequent')

X_train_copy[nominal_feature] = imputer_nom.fit_transform(X_train_copy[nominal_feature])
X_test_copy[nominal_feature] = imputer_nom.transform(X_test_copy[nominal_feature])

onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_onehoten = onehot_encoder.fit_transform(X_train_copy[nominal_feature])
X_test_onehoten = onehot_encoder.transform(X_train_copy[nominal_feature])

#X_train_onehoten.shape

#X_train_onehot

X_train_copy = pd.concat([X_train_copy.reset_index(drop=True),
                     pd.DataFrame(X_train_onehoten, columns=onehot_encoder.get_feature_names_out(['marital','contact','month','day_of_week','poutcome','default','housing','loan']))], axis=1)
X_train_copy = X_train_copy.drop(['marital','contact','month','day_of_week','poutcome','default','housing','loan'], axis=1) # ข้อมูลสุดท้ายที่ได้จะต้องเป็นตัวเลขเท่านั้น

X_train_copy

,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes
0,-1.338032,1.0,228,-0.564061,0.196989,-0.348214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0.673261,4.0,352,0.157272,0.196989,-0.348214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,-0.954928,4.0,102,-0.564061,0.196989,1.686930,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,-0.763377,2.0,39,-0.203395,0.196989,-0.348214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,-0.859152,4.0,81,0.517938,0.196989,-0.348214,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.859152,3.0,166,-0.564061,0.196989,-0.348214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
28827,0.673261,4.0,106,-0.564061,0.196989,-0.348214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
28828,2.205675,1.0,510,-0.203395,0.196989,1.686930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
28829,0.002830,4.0,172,0.157272,0.196989,-0.348214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [ ]:
X_test_copy = pd.concat([X_test_copy.reset_index(drop=True),
                     pd.DataFrame(X_test_onehoten, columns=onehot_encoder.get_feature_names_out(['marital','contact','month','day_of_week','poutcome','default','housing','loan']))], axis=1)
X_test_copy = X_test_copy.drop(['marital','contact','month','day_of_week','poutcome','default','housing','loan'], axis=1)

X_test_copy

,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes
0,-0.571825,1.0,141.0,-0.203395,0.196989,-0.348214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.343692,4.0,832.0,5.567270,0.196989,-0.348214,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,-1.146480,4.0,430.0,0.517938,0.196989,-0.348214,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,-0.954928,2.0,83.0,0.878605,0.196989,-0.348214,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,-0.380273,4.0,378.0,-0.203395,0.196989,-0.348214,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
28827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
28828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
28829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

First image [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

Number of training documents: 11314
Number of test documents: 7532
Categories: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

SyntaxError: invalid syntax (ipython-input-2984329282.py, line 11)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


